# Homework 7 Template

Rixin Li & G . Besla



In [ ]:
 # Make edits where instructed - look for "****", which indicates where you need to add code. 

In [5]:
# import necessary modules
# numpy provides powerful multi-dimensional arrays to hold and manipulate data
import numpy as np
# matplotlib provides powerful functions for plotting figures
import matplotlib.pyplot as plt
# astropy provides unit system and constants for astronomical calculations
import astropy.units as u
import astropy.constants as const
# import Latex module so we can display the results with symbols
from IPython.display import Latex
%matplotlib inline

# **** import CenterOfMass to determine the COM pos/vel of M33
import sys
import os
# Change path to homework4 where the class CenterOfMass is
module_path = r'C:\Users\kietb\OneDrive\Desktop\Suffering\Undergrad\ASTR400B\Homeworks\Homework4'

# Add the directory to sys.path
sys.path.append(module_path)

# Import CenterOfMass
from CenterOfMass import CenterOfMass

In [8]:
# **** import the GalaxyMass to determine the mass of M31 for each component
# Change path to homework3 where GalaxyMass is
module_path = r'C:\Users\kietb\OneDrive\Desktop\Suffering\Undergrad\ASTR400B\Homeworks\Homework3'

# Add the directory to sys.path
sys.path.append(module_path)

# Import ComponentMass
from GalaxyMass import ComponentMass


   Galaxy Name  Halo Mass (10e12 M_sun)  ...  Total Mass (10e12 M_sun)  f_bar
0           MW                    1.975  ...                     2.060  0.041
1          M31                    1.921  ...                     2.060  0.067
2          M33                    0.187  ...                     0.196  0.046
3  Local Group                    4.083  ...                     4.316  0.054

[4 rows x 6 columns]


# M33AnalyticOrbit

In [ ]:
class M33AnalyticOrbit:
    """ Calculate the analytical orbit of M33 around M31 """
    
    def __init__(self, filename): # **** add inputs
        """ Initialize the orbit calculation.

        PARAMETERS
        ----------
        filename: `str`
            Name of the output file to store the orbit data.

        INITIALIZED QUANTITIES
        ----------------------
        G: `float`
            Gravitational constant in kpc^3 / (Msun Gyr^2)
        r0: `array`
            Initial relative position vector of M33 to M31 (kpc)
        v0: `array`
            Initial relative velocity vector of M33 to M31 (km/s)
        rdisk: `float`
            Scale length of M31's disk (kpc)
        Mdisk: `float`
            Mass of M31's disk (Msun)
        rbulge: `float`
            Scale length of M31's bulge (kpc)
        Mbulge: `float`
            Mass of M31's bulge (Msun)
        rhalo: `float`
            Hernquist scale length of M31's halo (kpc)
        Mhalo: `float`
            Mass of M31's halo (Msun)
        """

        ### get the gravitational constant (the value is 4.498502151575286e-06)
        self.G = const.G.to(u.kpc**3/u.Msun/u.Gyr**2).value
        
        ### **** store the output file name
        self.filename = filename  
        
        ### get the current pos/vel of M33 
        # **** create an instance of the  CenterOfMass class for M33 
        M33_COM = CenterOfMass("M33_000.txt", 2)

        # **** store the position VECTOR of the M33 COM (.value to get rid of units)
        M33_pos = M33_COM.COM_P(0.1).value

        # **** store the velocity VECTOR of the M33 COM (.value to get rid of units)
        M33_vel = M33_COM.COM_V(M33_pos).value
        
        ### get the current pos/vel of M31 
        # **** create an instance of the  CenterOfMass class for M31 
        M31_COM = CenterOfMass("M31_000.txt", 2)
        
        # **** store the position VECTOR of the M31 COM (.value to get rid of units)
        M31_pos = M31_COM.COM_P(0.1).value

        # **** store the velocity VECTOR of the M31 COM (.value to get rid of units)
        M31_vel = M31_COM.COM_V(M31_pos).value
        
        ### store the DIFFERENCE between the vectors posM33 - posM31
        # **** create two VECTORs self.r0 and self.v0 and have them be the
        self.r0 = M33_pos - M31_pos
        self.v0 = M33_vel - M31_vel
        # relative position and velocity VECTORS of M33
        
        ### get the mass of each component in M31 
        ### disk
        # **** self.rdisk = scale length (no units)
        self.rdisk = 5

        # **** self.Mdisk set with ComponentMass function. Remember to *1e12 to get the right units. Use the right ptype
        self.Mdisk = ComponentMass("M31_000.txt", 2) * 1e12 

        ### bulge
        # **** self.rbulge = set scale length (no units)
        self.rbulge = 1

        # **** self.Mbulge  set with ComponentMass function. Remember to *1e12 to get the right units Use the right ptype
        self.Mbulge = ComponentMass("M31_000.txt", 3) * 1e12  

        # Halo
        # **** self.rhalo = set scale length from HW5 (no units)
        self.rhalo = 61 

        # **** self.Mhalo set with ComponentMass function. Remember to *1e12 to get the right units. Use the right ptype
        self.Mhalo = ComponentMass("M31_000.txt", 1) * 1e12  
    
    
    def HernquistAccel(self, M, ra, r): # it is easiest if you take as an input the position VECTOR 
        """
        Compute the acceleration due to a Hernquist potential.

        PARAMETERS
        ----------
        M: `float`
            Mass of the component (Msun)
        ra: `float`
            Scale length of the Hernquist profile (kpc)
        r: `array`
            Position vector of the object (kpc)

        OUTPUTS
        -------
        Hern: `array`
            Acceleration vector from the Hernquist profile (kpc/Gyr^2)
        """
        
        ### **** Store the magnitude of the position vector
        rmag = np.linalg.norm(r)
        
        ### *** Store the Acceleration
        Hern = -self.G * M / (rmag * (ra + rmag)**2) * r  #follow the formula in the HW instructions
        # NOTE: we want an acceleration VECTOR so you need to make sure that in the Hernquist equation you 
        # use  -G*M/(rmag *(ra + rmag)**2) * r --> where the last r is a VECTOR 
        
        return Hern
    
    
    
    def MiyamotoNagaiAccel(self, M, rd, r):# it is easiest if you take as an input a position VECTOR  r 
        """
        Compute the acceleration using a Miyamoto-Nagai profile for disk potential.

        PARAMETERS
        ----------
        M: `float`
            Mass of the disk (Msun)
        rd: `float`
            Scale length of the disk (kpc)
        r: `array`
            Position vector of the object (kpc)

        OUTPUTS
        -------
        accel: `array`
            Acceleration vector from the Miyamoto-Nagai profile (kpc/Gyr^2)
        """

        # Define position vector by its components
        x, y, z = r

        # Compute the radial distance in the disk plane (cylindrical radius)
        R = np.sqrt(x**2 + y**2)

        # Define the scale height of the disk, zd, as rd/5.0
        zd = rd / 5.0

        # Compute the B parameter for the potential
        B = rd + np.sqrt(z**2 + zd**2)

        ### Acceleration **** follow the formula in the HW instructions
        # AGAIN note that we want a VECTOR to be returned  (see Hernquist instructions)
        # this can be tricky given that the z component is different than in the x or y directions. 
        # we can deal with this by multiplying the whole thing by an extra array that accounts for the 
        # differences in the z direction:
        #  multiply the whle thing by :   np.array([1,1,ZSTUFF]) 
        # where ZSTUFF are the terms associated with the z direction
        
        # Compute the acceleration vector (before applying z direction corrections)
        accel = -self.G * M / (R**2 + B**2)**1.5 * r
      
        # Applying the z direction corrections
        accel *= np.array([1, 1, B / np.sqrt(z**2 + zd**2)])
        
        
       
        return accel
        # the np.array allows for a different value for the z component of the acceleration
     
    
    def M31Accel("****"): # input should include the position vector, r
                       """ **** ADD COMMENTS """

        ### Call the previous functions for the halo, bulge and disk
        # **** these functions will take as inputs variable we defined in the initialization of the class like 
        # self.rdisk etc. 
            
            # return the SUM of the output of the acceleration functions - this will return a VECTOR 
        return 
    
    
    
    def LeapFrog("****"): # take as input r and v, which are VECTORS. Assume it is ONE vector at a time
                               """ **** ADD COMMENTS """
        
        # predict the position at the next half timestep
        rhalf = 
        
        # predict the final velocity at the next timestep using the acceleration field at the rhalf position 
        vnew = 
        
        # predict the final position using the average of the current velocity and the final velocity
        # this accounts for the fact that we don't know how the speed changes from the current timestep to the 
        # next, so we approximate it using the average expected speed over the time interval dt. 
        rnew = 
        
        return # **** return the new position and velcoity vectors
    
    
    
    def OrbitIntegration(self, t0, dt, tmax):
         """ **** ADD COMMENTS """

        # initialize the time to the input starting time
        t = "****"
        
        # initialize an empty array of size :  rows int(tmax/dt)+2  , columns 7
        orbit = np.zeros("****")
        
        # initialize the first row of the orbit
        orbit[0] = t0, *tuple(self.r0), *tuple(self.v0)
        # this above is equivalent to 
        # orbit[0] = t0, self.r0[0], self.r0[1], self.r0[2], self.v0[0], self.v0[1], self.v0[2]
        
        
        # initialize a counter for the orbit.  
        i = 1 # since we already set the 0th values, we start the counter at 1
        
        # start the integration (advancing in time steps and computing LeapFrog at each step)
        while ("****"):  # as long as t has not exceeded the maximal time 
            
            # **** advance the time by one timestep, dt
           
            # **** store the new time in the first column of the ith row
            orbit["****"] =
            
            # ***** advance the position and velocity using the LeapFrog scheme
            # remember that LeapFrog returns a position vector and a velocity vector  
            # as an example, if a function returns three vectors you would call the function and store 
            # the variable like:     a,b,c = function(input)
            
         
    
            # ****  store the new position vector into the columns with indexes 1,2,3 of the ith row of orbit
            # TIP:  if you want columns 5-7 of the Nth row of an array called A, you would write : 
            # A[n, 5:8] 
            # where the syntax is row n, start at column 5 and end BEFORE column 8
            
            
            # ****  store the new position vector into the columns with indexes 1,2,3 of the ith row of orbit
            
            
            # **** update counter i , where i is keeping track of the number of rows (i.e. the number of time steps)
        
        
        
        # write the data to a file
        np.savetxt(self.filename, orbit, fmt = "%11.3f"*7, comments='#', 
                   header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                   .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))
        
        # there is no return function
        
        

IndentationError: unindent does not match any outer indentation level (<string>, line 178)